10.1 GroupBy Mechanics

In [4]:
import pandas as pd
import numpy as np


df = pd.DataFrame({'key1': ['a', 'a', 'b', 'b', 'a'],
                   'key2': ['one', 'two', 'one', 'two', 'one'],
                   'data1': np.random.randn(5),
                   'data2': np.random.randn(5)})
df
grouped = df['data1'].groupby(df['key1'])
grouped
grouped.mean()


key1
a    0.240843
b    0.104298
Name: data1, dtype: float64

In [5]:
means = df['data1'].groupby([df['key1'], df['key2']]).mean()
means

key1  key2
a     one     0.549970
      two    -0.377410
b     one     0.488064
      two    -0.279469
Name: data1, dtype: float64

In [6]:
means.unstack()

key2,one,two
key1,,
a,0.549970,-0.377410
b,0.488064,-0.279469


In [7]:
states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])
years = np.array([2005, 2005, 2006, 2005, 2006])

df['data1'].groupby([states, years]).mean()

California  2005   -0.377410
            2006    0.488064
Ohio        2005    0.409349
            2006    0.001773
Name: data1, dtype: float64

In [13]:
df.groupby('key1').mean(numeric_only=True)

,data1,data2
key1,,
a,0.240843,-1.694155
b,0.104298,1.012113


In [11]:
df.groupby(['key1', 'key2']).mean()

data1     data2
key1 key2                    
a    one   0.549970 -1.010480
     two  -0.377410 -3.061505
b    one   0.488064  2.202667
     two  -0.279469 -0.178441

In [14]:
df.groupby(['key1', 'key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

In [15]:
# Iterating Over Groups

for name, group in df.groupby('key1'):
    print(name)
    print(group)

a
  key1 key2     data1     data2
0    a  one  1.098167 -0.835256
1    a  two -0.377410 -3.061505
4    a  one  0.001773 -1.185705
b
  key1 key2     data1     data2
2    b  one  0.488064  2.202667
3    b  two -0.279469 -0.178441


In [17]:
for (k1, k2), group in df.groupby(['key1', 'key2']):
    print((k1, k2))
    print(group)

('a', 'one')
  key1 key2     data1     data2
0    a  one  1.098167 -0.835256
4    a  one  0.001773 -1.185705
('a', 'two')
  key1 key2    data1     data2
1    a  two -0.37741 -3.061505
('b', 'one')
  key1 key2     data1     data2
2    b  one  0.488064  2.202667
('b', 'two')
  key1 key2     data1     data2
3    b  two -0.279469 -0.178441


In [18]:
pieces = dict(list(df.groupby('key1')))
pieces['b']

,key1,key2,data1,data2
2,b,one,0.488064,2.202667
3,b,two,-0.279469,-0.178441


In [20]:
df.dtypes
for dtype, group in grouped:
    print(dtype)
    print(group)

a
0    1.098167
1   -0.377410
4    0.001773
Name: data1, dtype: float64
b
2    0.488064
3   -0.279469
Name: data1, dtype: float64


In [26]:
# Selecting a Column or Subset of Columns

df.groupby(['key1', 'key2'])[['data2']].mean()

data2
key1 key2          
a    one  -1.010480
     two  -3.061505
b    one   2.202667
     two  -0.178441

In [29]:
s_grouped = df.groupby(['key1', 'key2'])['data2']
s_grouped
s_grouped.mean()

key1  key2
a     one    -1.010480
      two    -3.061505
b     one     2.202667
      two    -0.178441
Name: data2, dtype: float64

In [21]:
# Grouping with Dicts and Series

people = pd.DataFrame(np.random.randn(5, 5),
                      columns=['a', 'b', 'c', 'd', 'e'],
                      index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])
people.iloc[2:3, [1, 2]] = np.nan
people

,a,b,c,d,e
Joe,0.497707,-1.869093,0.163027,-0.060932,0.719777
Steve,-0.134266,0.350190,-0.149868,-0.251269,-1.036591
Wes,0.116678,NaN,NaN,-0.352340,1.069841
Jim,1.807934,-0.489560,0.562145,-0.135978,-1.046192
Travis,0.131337,-0.264420,0.061621,0.684875,-0.060339


In [23]:
mapping = {'a': 'red', 'b': 'red', 'c': 'blue', 'd': 'blue', 'e': 'red', 'f': 'orange'}
by_column = people.T.groupby(mapping)
by_column.sum()

,Joe,Steve,Wes,Jim,Travis
blue,0.102095,-0.401137,-0.352340,0.426167,0.746496
red,-0.651609,-0.820667,1.186519,0.272183,-0.193422


In [25]:
map_series = pd.Series(mapping)
map_series
people.T.groupby(map_series).count()

,Joe,Steve,Wes,Jim,Travis
blue,2,2,1,2,2
red,3,3,2,3,3


In [30]:
# Grouping with Functions

people.groupby(len).sum()

,a,b,c,d,e
3,2.422320,-2.358653,0.725171,-0.549249,0.743426
5,-0.134266,0.350190,-0.149868,-0.251269,-1.036591
6,0.131337,-0.264420,0.061621,0.684875,-0.060339


In [31]:
key_list = ['one', 'one', 'one', 'two', 'two']
people.groupby([len, key_list]).min()

a         b         c         d         e
3 one  0.116678 -1.869093  0.163027 -0.352340  0.719777
  two  1.807934 -0.489560  0.562145 -0.135978 -1.046192
5 one -0.134266  0.350190 -0.149868 -0.251269 -1.036591
6 two  0.131337 -0.264420  0.061621  0.684875 -0.060339

In [32]:
# Grouping by Index Levels

columns = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP', 'JP'],
                                     [1, 3, 5, 1, 3]],
                                     names=['city', 'tenor'])
hier_df = pd.DataFrame(np.random.randn(4, 5), columns=columns)
hier_df

city         US                            JP          
tenor         1         3         5         1         3
0      0.474790 -1.294720  0.857986 -1.352184 -0.352745
1      1.318117  0.326768  1.865385  0.637871 -1.397068
2     -0.565447 -0.755753  0.111416 -0.305201 -0.310356
3      0.233398 -0.185457  0.769585  0.206446 -0.147800

In [35]:
hier_df.T.groupby(level='city').count()

,0,1,2,3
city,,,,
JP,2,2,2,2
US,3,3,3,3


10.2 Data Aggregation

In [38]:
df
grouped = df.groupby('key1')
grouped['data1'].quantile(0.9)

key1
a    0.878888
b    0.411311
Name: data1, dtype: float64

In [40]:
def peak_to_peak(arr):
    return arr.max() - arr.min()

grouped[grouped.obj.select_dtypes(include='number').columns].agg(peak_to_peak)

,data1,data2
key1,,
a,1.475577,2.226249
b,0.767533,2.381108


In [41]:
grouped.describe()

data1                                                              \
     count      mean       std       min       25%       50%       75%   
key1                                                                     
a      3.0  0.240843  0.766288 -0.377410 -0.187818  0.001773  0.549970   
b      2.0  0.104298  0.542728 -0.279469 -0.087586  0.104298  0.296181   

               data2                                                    \
           max count      mean       std       min       25%       50%   
key1                                                                     
a     1.098167   3.0 -1.694155  1.197054 -3.061505 -2.123605 -1.185705   
b     0.488064   2.0  1.012113  1.683698 -0.178441  0.416836  1.012113   

                         
          75%       max  
key1                     
a    -1.01048 -0.835256  
b     1.60739  2.202667

In [45]:
# Column-Wise and Multiple Function Application

tips = pd.read_csv('examples/napiwki.csv')
tips
tips['tip_pct'] = tips['napiwek'] / tips['rachunek']
tips[:6]

,rachunek,napiwek,palacz,dzień,pora,wielkość,tip_pct
0,16.99,1.01,nie,niedz.,kolacja,2,0.059447
1,10.34,1.66,nie,niedz.,kolacja,3,0.160542
2,21.01,3.50,nie,niedz.,kolacja,3,0.166587
3,23.68,3.31,nie,niedz.,kolacja,2,0.139780
4,24.59,3.61,nie,niedz.,kolacja,4,0.146808
5,25.29,4.71,nie,niedz.,kolacja,4,0.186240


In [46]:
grouped = tips.groupby(['dzień', 'palacz'])
grouped_pct = grouped['tip_pct']
grouped_pct.agg('mean')

dzień   palacz
czw.    nie       0.160298
        tak       0.163863
niedz.  nie       0.160113
        tak       0.187250
pt.     nie       0.151650
        tak       0.174783
sob.    nie       0.158048
        tak       0.147906
Name: tip_pct, dtype: float64

In [47]:
grouped_pct.agg(['mean', 'std', peak_to_peak])

mean       std  peak_to_peak
dzień  palacz                                  
czw.   nie     0.160298  0.038774      0.193350
       tak     0.163863  0.039389      0.151240
niedz. nie     0.160113  0.042347      0.193226
       tak     0.187250  0.154134      0.644685
pt.    nie     0.151650  0.028123      0.067349
       tak     0.174783  0.051293      0.159925
sob.   nie     0.158048  0.039767      0.235193
       tak     0.147906  0.061375      0.290095

In [50]:
grouped_pct.agg([('foo', 'mean'), ('bar', 'std')])

foo       bar
dzień  palacz                    
czw.   nie     0.160298  0.038774
       tak     0.163863  0.039389
niedz. nie     0.160113  0.042347
       tak     0.187250  0.154134
pt.    nie     0.151650  0.028123
       tak     0.174783  0.051293
sob.   nie     0.158048  0.039767
       tak     0.147906  0.061375

In [53]:
functions = ['count', 'mean', 'max']
result = grouped[['tip_pct', 'rachunek']].agg(functions)
result

tip_pct                     rachunek                  
                count      mean       max    count       mean    max
dzień  palacz                                                       
czw.   nie         45  0.160298  0.266312       45  17.113111  41.19
       tak         17  0.163863  0.241255       17  19.190588  43.11
niedz. nie         57  0.160113  0.252672       57  20.506667  48.17
       tak         19  0.187250  0.710345       19  24.120000  45.35
pt.    nie          4  0.151650  0.187735        4  18.420000  22.75
       tak         15  0.174783  0.263480       15  16.813333  40.17
sob.   nie         45  0.158048  0.291990       45  19.661778  48.33
       tak         42  0.147906  0.325733       42  21.276667  50.81

In [54]:
result['tip_pct']

count      mean       max
dzień  palacz                           
czw.   nie        45  0.160298  0.266312
       tak        17  0.163863  0.241255
niedz. nie        57  0.160113  0.252672
       tak        19  0.187250  0.710345
pt.    nie         4  0.151650  0.187735
       tak        15  0.174783  0.263480
sob.   nie        45  0.158048  0.291990
       tak        42  0.147906  0.325733

In [56]:
ftuples = [('Durchschnitt', 'mean'), ('Abweichung', 'var')]
grouped[['tip_pct', 'rachunek']].agg(ftuples)

tip_pct                rachunek            
              Durchschnitt Abweichung Durchschnitt  Abweichung
dzień  palacz                                                 
czw.   nie        0.160298   0.001503    17.113111   59.625081
       tak        0.163863   0.001551    19.190588   69.808518
niedz. nie        0.160113   0.001793    20.506667   66.099980
       tak        0.187250   0.023757    24.120000  109.046044
pt.    nie        0.151650   0.000791    18.420000   25.596333
       tak        0.174783   0.002631    16.813333   82.562438
sob.   nie        0.158048   0.001581    19.661778   79.908965
       tak        0.147906   0.003767    21.276667  101.387535

In [60]:
grouped.agg({'napiwek': 'max', 'wielkość': 'sum'})

napiwek  wielkość
dzień  palacz                   
czw.   nie        6.70       112
       tak        5.00        40
niedz. nie        6.00       167
       tak        6.50        49
pt.    nie        3.50         9
       tak        4.73        31
sob.   nie        9.00       115
       tak       10.00       104

In [61]:
grouped.agg({'tip_pct': ['min', 'max', 'mean', 'std'], 'wielkość': 'sum'})

tip_pct                               wielkość
                    min       max      mean       std      sum
dzień  palacz                                                 
czw.   nie     0.072961  0.266312  0.160298  0.038774      112
       tak     0.090014  0.241255  0.163863  0.039389       40
niedz. nie     0.059447  0.252672  0.160113  0.042347      167
       tak     0.065660  0.710345  0.187250  0.154134       49
pt.    nie     0.120385  0.187735  0.151650  0.028123        9
       tak     0.103555  0.263480  0.174783  0.051293       31
sob.   nie     0.056797  0.291990  0.158048  0.039767      115
       tak     0.035638  0.325733  0.147906  0.061375      104

In [63]:
# Returning Aggregated Data Without Row Indexes

tips.groupby(['dzień', 'palacz'], as_index=False).mean(numeric_only=True)

,dzień,palacz,rachunek,napiwek,wielkość,tip_pct
0,czw.,nie,17.113111,2.673778,2.488889,0.160298
1,czw.,tak,19.190588,3.030000,2.352941,0.163863
2,niedz.,nie,20.506667,3.167895,2.929825,0.160113
3,niedz.,tak,24.120000,3.516842,2.578947,0.187250
4,pt.,nie,18.420000,2.812500,2.250000,0.151650
5,pt.,tak,16.813333,2.714000,2.066667,0.174783
6,sob.,nie,19.661778,3.102889,2.555556,0.158048
7,sob.,tak,21.276667,2.875476,2.476190,0.147906


10.3 Apply: General split-apply-combine

In [64]:
def top(df, n=5, column='tip_pct'):
    return df.sort_values(by=column)[-n:]

top(tips, n=6)

,rachunek,napiwek,palacz,dzień,pora,wielkość,tip_pct
109,14.31,4.00,tak,sob.,kolacja,2,0.279525
183,23.17,6.50,tak,niedz.,kolacja,4,0.280535
232,11.61,3.39,nie,sob.,kolacja,2,0.291990
67,3.07,1.00,tak,sob.,kolacja,1,0.325733
178,9.60,4.00,tak,niedz.,kolacja,2,0.416667
172,7.25,5.15,tak,niedz.,kolacja,2,0.710345


In [67]:
tips.groupby('palacz').apply(top, include_groups=False)

rachunek  napiwek   dzień     pora  wielkość   tip_pct
palacz                                                            
nie    88      24.71     5.85    czw.    obiad         2  0.236746
       185     20.69     5.00  niedz.  kolacja         5  0.241663
       51      10.29     2.60  niedz.  kolacja         2  0.252672
       149      7.51     2.00    czw.    obiad         2  0.266312
       232     11.61     3.39    sob.  kolacja         2  0.291990
tak    109     14.31     4.00    sob.  kolacja         2  0.279525
       183     23.17     6.50  niedz.  kolacja         4  0.280535
       67       3.07     1.00    sob.  kolacja         1  0.325733
       178      9.60     4.00  niedz.  kolacja         2  0.416667
       172      7.25     5.15  niedz.  kolacja         2  0.710345

In [70]:
tips.groupby(['palacz', 'dzień']).apply(top, n=1, column='rachunek', include_groups=False)

rachunek  napiwek     pora  wielkość   tip_pct
palacz dzień                                                     
nie    czw.   142     41.19     5.00    obiad         5  0.121389
       niedz. 156     48.17     5.00  kolacja         6  0.103799
       pt.    94      22.75     3.25  kolacja         2  0.142857
       sob.   212     48.33     9.00  kolacja         4  0.186220
tak    czw.   197     43.11     5.00    obiad         4  0.115982
       niedz. 182     45.35     3.50  kolacja         3  0.077178
       pt.    95      40.17     4.73  kolacja         4  0.117750
       sob.   170     50.81    10.00  kolacja         3  0.196812

In [71]:
result = tips.groupby('palacz')['tip_pct'].describe()
result

,count,mean,std,min,25%,50%,75%,max
palacz,,,,,,,,
nie,151.0,0.159328,0.039910,0.056797,0.136906,0.155625,0.185014,0.291990
tak,93.0,0.163196,0.085119,0.035638,0.106771,0.153846,0.195059,0.710345


In [72]:
result.unstack('palacz')

       palacz
count  nie       151.000000
       tak        93.000000
mean   nie         0.159328
       tak         0.163196
std    nie         0.039910
       tak         0.085119
min    nie         0.056797
       tak         0.035638
25%    nie         0.136906
       tak         0.106771
50%    nie         0.155625
       tak         0.153846
75%    nie         0.185014
       tak         0.195059
max    nie         0.291990
       tak         0.710345
dtype: float64

In [74]:
# Suppressing the Group Keys

tips.groupby('palacz', group_keys=False).apply(top, include_groups=False)

,rachunek,napiwek,dzień,pora,wielkość,tip_pct
88,24.71,5.85,czw.,obiad,2,0.236746
185,20.69,5.00,niedz.,kolacja,5,0.241663
51,10.29,2.60,niedz.,kolacja,2,0.252672
149,7.51,2.00,czw.,obiad,2,0.266312
232,11.61,3.39,sob.,kolacja,2,0.291990
109,14.31,4.00,sob.,kolacja,2,0.279525
183,23.17,6.50,niedz.,kolacja,4,0.280535
67,3.07,1.00,sob.,kolacja,1,0.325733
178,9.60,4.00,niedz.,kolacja,2,0.416667
172,7.25,5.15,niedz.,kolacja,2,0.710345


In [75]:
# Quantile and Bucket Analysis

frame = pd.DataFrame({'data1': np.random.randn(1000),
                      'data2': np.random.randn(1000)})
quartiles = pd.cut(frame.data1, 4)
quartiles[:10]

0    (-1.579, 0.327]
1    (-1.579, 0.327]
2    (-1.579, 0.327]
3     (0.327, 2.234]
4    (-1.579, 0.327]
5     (0.327, 2.234]
6     (0.327, 2.234]
7     (0.327, 2.234]
8    (-1.579, 0.327]
9    (-1.579, 0.327]
Name: data1, dtype: category
Categories (4, interval[float64, right]): [(-3.493, -1.579] < (-1.579, 0.327] < (0.327, 2.234] < (2.234, 4.14]]

In [77]:
def get_stats(group):
    return {'min': group.min(), 'max': group.max(), 'count': group.count(), 'mean': group.mean()}

grouped = frame.data2.groupby(quartiles, observed=False)
grouped.apply(get_stats).unstack()

,min,max,count,mean
data1,,,,
"(-3.493, -1.579]",-2.091311,1.952777,56.0,0.265965
"(-1.579, 0.327]",-3.002876,2.760429,570.0,0.029890
"(0.327, 2.234]",-3.201347,2.720848,356.0,0.030729
"(2.234, 4.14]",-1.822711,1.427156,18.0,0.045123


In [79]:
grouping = pd.qcut(frame.data1, 10, labels=False)
grouped = frame.data2.groupby(grouping)
grouped.apply(get_stats).unstack()

,min,max,count,mean
data1,,,,
0,-2.091311,1.952777,100.0,0.036594
1,-2.787321,2.760429,100.0,-0.045229
2,-2.040690,2.457407,100.0,0.163003
3,-3.002876,2.490260,100.0,-0.098438
4,-2.603770,2.683276,100.0,0.184262
5,-1.801649,2.303693,100.0,0.192769
6,-3.201347,2.314948,100.0,-0.138576
7,-2.689880,2.720848,100.0,0.117643
8,-2.310798,2.427663,100.0,-0.028291


In [81]:
# Example: Filling Missing Values with Group-Specific Values

s = pd.Series(np.random.randn(6))
s[::2] = np.nan
s
s.fillna(s.mean())

0   -0.229999
1    0.003482
2   -0.229999
3   -0.061813
4   -0.229999
5   -0.631666
dtype: float64

In [87]:
states = ['Ohio', 'New York', 'Vermont', 'Florida', 'Oregon', 'Nevada', 'California', 'Idaho']
group_key = ['East'] * 4 + ['West'] * 4
data = pd.Series(np.random.randn(8), index=states)
data

Ohio         -1.430110
New York      0.259731
Vermont       0.672780
Florida      -0.748498
Oregon        1.094719
Nevada        0.876146
California   -0.781240
Idaho        -0.084502
dtype: float64

In [84]:
data[['Vermont', 'Nevada', 'Idaho']] = np.nan
data
data.groupby(group_key).mean()

East   -0.235071
Wers    0.033369
dtype: float64

In [85]:
fill_mean = lambda g: g.fillna(g.mean())
data.groupby(group_key).apply(fill_mean)

East  Ohio         -0.488877
      New York      0.390004
      Vermont      -0.235071
      Florida      -0.606341
Wers  Oregon        0.160431
      Nevada        0.033369
      California   -0.093693
      Idaho         0.033369
dtype: float64

In [88]:
fill_values = {'East': 0.5, 'West': -1}
fill_func = lambda g: g.fillna(fill_values[g.name])
data.groupby(group_key).apply(fill_func)

East  Ohio         -1.430110
      New York      0.259731
      Vermont       0.672780
      Florida      -0.748498
West  Oregon        1.094719
      Nevada        0.876146
      California   -0.781240
      Idaho        -0.084502
dtype: float64

In [89]:
# Example: Random Sampling and Permutation

# Suit designations: H - hearts, S - spades, C - clubs, D - diamonds
suits = ['H', 'S', 'C', 'D']
card_val = (list(range(1, 11)) + [10] * 3) * 4
base_names = ['A'] + list(range(2, 11)) + ['J', 'K', 'Q']
cards = []
for suit in ['H', 'S', 'C', 'D']:
    cards.extend(str(num) + suit for num in base_names)

deck = pd.Series(card_val, index=cards)

deck[:13]


AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
KH     10
QH     10
dtype: int64

In [90]:
def draw(deck, n=5):
    return deck.sample(n)

draw(deck)

6D      6
QH     10
AC      1
10C    10
8D      8
dtype: int64

In [91]:
get_suit = lambda card: card[-1]
deck.groupby(get_suit).apply(draw, n=2)

C  QC    10
   3C     3
D  9D     9
   JD    10
H  QH    10
   2H     2
S  8S     8
   6S     6
dtype: int64

In [92]:
deck.groupby(get_suit, group_keys=False).apply(draw, n=2)

JC     10
10C    10
8D      8
10D    10
7H      7
KH     10
10S    10
9S      9
dtype: int64

In [94]:
# Example: Group Weighted Average and Correlation

df = pd.DataFrame({'category': ['a', 'a', 'a', 'a', 'b', 'b', 'b', 'b'],
                   'data': np.random.randn(8),
                   'weights': np.random.rand(8)})
df

,category,data,weights
0,a,-0.071566,0.033366
1,a,1.630250,0.880527
2,a,-0.301642,0.296527
3,a,0.018666,0.706009
4,b,0.564276,0.061909
5,b,0.426752,0.075572
6,b,-0.577657,0.966814
7,b,-0.897656,0.831031


In [96]:
grouped = df.groupby('category')
get_wavg = lambda g: np.average(g['data'], weights=g['weights'])
grouped.apply(get_wavg, include_groups=False)

category
a    0.707996
b   -0.639315
dtype: float64

In [102]:
close_px = pd.read_csv("examples/stock_px.csv", parse_dates=True, index_col=0)
close_px.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2214 entries, 2003-01-02 to 2011-10-14
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AAPL    2214 non-null   float64
 1   MSFT    2214 non-null   float64
 2   XOM     2214 non-null   float64
 3   SPX     2214 non-null   float64
dtypes: float64(4)
memory usage: 86.5 KB


In [103]:
close_px[-4:]

,AAPL,MSFT,XOM,SPX
2011-10-11,400.29,27.00,76.27,1195.54
2011-10-12,402.19,26.96,77.16,1207.25
2011-10-13,408.43,27.18,76.37,1203.66
2011-10-14,422.00,27.27,78.11,1224.58


In [104]:
spx_corr = lambda x: x.corrwith(x['SPX'])
rets = close_px.pct_change().dropna()
get_year = lambda x: x.year
by_year = rets.groupby(get_year)
by_year.apply(spx_corr)

,AAPL,MSFT,XOM,SPX
2003,0.541124,0.745174,0.661265,1.0
2004,0.374283,0.588531,0.557742,1.0
2005,0.467540,0.562374,0.631010,1.0
2006,0.428267,0.406126,0.518514,1.0
2007,0.508118,0.658770,0.786264,1.0
2008,0.681434,0.804626,0.828303,1.0
2009,0.707103,0.654902,0.797921,1.0
2010,0.710105,0.730118,0.839057,1.0
2011,0.691931,0.800996,0.859975,1.0


In [105]:
by_year.apply(lambda g: g['AAPL'].corr(g['MSFT']))

2003    0.480868
2004    0.259024
2005    0.300093
2006    0.161735
2007    0.417738
2008    0.611901
2009    0.432738
2010    0.571946
2011    0.581987
dtype: float64

In [ ]:
# Example: Group-Wise Linear Regression

import statsmodels.api as sm


def regress(data, yvar=None, xvars=None):
    Y = data[yvar]
    X = data[xvars]
    X["intercept"] = 1.
    result = sm.OLS(Y, X).fit()
    return result.params

by_year.apply(regress, yvar="AAPL", xvars=["SPX"])

# https://github.com/statsmodels/statsmodels/issues/9584

ImportError: cannot import name '_lazywhere' from 'scipy._lib._util' (/Users/metju29/CursorProjects/data-learning/.venv/lib/python3.13/site-packages/scipy/_lib/_util.py)

10.4 Pivot Tables and Cross-Tabulation

In [125]:
num_cols = tips.select_dtypes(include='number').columns
tips.pivot_table(index=['dzień', 'palacz'], values=num_cols)

napiwek   rachunek   tip_pct  wielkość
dzień  palacz                                         
czw.   nie     2.673778  17.113111  0.160298  2.488889
       tak     3.030000  19.190588  0.163863  2.352941
niedz. nie     3.167895  20.506667  0.160113  2.929825
       tak     3.516842  24.120000  0.187250  2.578947
pt.    nie     2.812500  18.420000  0.151650  2.250000
       tak     2.714000  16.813333  0.174783  2.066667
sob.   nie     3.102889  19.661778  0.158048  2.555556
       tak     2.875476  21.276667  0.147906  2.476190

In [126]:
tips.pivot_table(['tip_pct', 'wielkość'], index=['pora', 'dzień'], columns='palacz')

tip_pct            wielkość          
palacz               nie       tak       nie       tak
pora    dzień                                         
kolacja czw.    0.159744       NaN  2.000000       NaN
        niedz.  0.160113  0.187250  2.929825  2.578947
        pt.     0.139622  0.165347  2.000000  2.222222
        sob.    0.158048  0.147906  2.555556  2.476190
obiad   czw.    0.160311  0.163863  2.500000  2.352941
        pt.     0.187735  0.188937  3.000000  1.833333

In [127]:
tips.pivot_table(['tip_pct', 'wielkość'], index=['pora', 'dzień'], columns='palacz', margins=True)

tip_pct                      wielkość                    
palacz               nie       tak       All       nie       tak       All
pora    dzień                                                             
kolacja czw.    0.159744       NaN  0.159744  2.000000       NaN  2.000000
        niedz.  0.160113  0.187250  0.166897  2.929825  2.578947  2.842105
        pt.     0.139622  0.165347  0.158916  2.000000  2.222222  2.166667
        sob.    0.158048  0.147906  0.153152  2.555556  2.476190  2.517241
obiad   czw.    0.160311  0.163863  0.161301  2.500000  2.352941  2.459016
        pt.     0.187735  0.188937  0.188765  3.000000  1.833333  2.000000
All             0.159328  0.163196  0.160803  2.668874  2.408602  2.569672

In [128]:
tips.pivot_table('tip_pct', index=['pora', 'palacz'], columns='dzień', aggfunc=len, margins=True)

dzień           czw.  niedz.   pt.  sob.  All
pora    palacz                               
kolacja nie      1.0    57.0   3.0  45.0  106
        tak      NaN    19.0   9.0  42.0   70
obiad   nie     44.0     NaN   1.0   NaN   45
        tak     17.0     NaN   6.0   NaN   23
All             62.0    76.0  19.0  87.0  244

In [129]:
tips.pivot_table('tip_pct', index=['pora', 'wielkość', 'palacz'], columns='dzień', aggfunc='mean', fill_value=0)

dzień                        czw.    niedz.       pt.      sob.
pora    wielkość palacz                                        
kolacja 1        nie     0.000000  0.000000  0.000000  0.137931
                 tak     0.000000  0.000000  0.000000  0.325733
        2        nie     0.159744  0.168859  0.139622  0.162705
                 tak     0.000000  0.207893  0.171297  0.148668
        3        nie     0.000000  0.152663  0.000000  0.154661
                 tak     0.000000  0.152660  0.000000  0.144995
        4        nie     0.000000  0.148143  0.000000  0.150096
                 tak     0.000000  0.193370  0.117750  0.124515
        5        nie     0.000000  0.206928  0.000000  0.000000
                 tak     0.000000  0.065660  0.000000  0.106572
        6        nie     0.000000  0.103799  0.000000  0.000000
obiad   1        nie     0.181728  0.000000  0.000000  0.000000
                 tak     0.000000  0.000000  0.223776  0.000000
        2        nie     0.166005  0.000000  0.000000  0.000000
                 tak     0.158843  0.000000  0.181969  0.000000
        3        nie     0.084246  0.000000  0.187735  0.000000
                 tak     0.204952  0.000000  0.000000  0.000000
        4        nie     0.138919  0.000000  0.000000  0.000000
                 tak     0.155410  0.000000  0.000000  0.000000
        5        nie     0.121389  0.000000  0.000000  0.000000
        6        nie     0.173706  0.000000  0.000000  0.000000

In [131]:
# Cross-Tabulations: Crosstab

pd.crosstab([tips.pora, tips.dzień], tips.palacz, margins=True)

palacz          nie  tak  All
pora    dzień                
kolacja czw.      1    0    1
        niedz.   57   19   76
        pt.       3    9   12
        sob.     45   42   87
obiad   czw.     44   17   61
        pt.       1    6    7
All             151   93  244